In [104]:
import cv2

In [105]:
from PIL import Image

In [106]:
import numpy as np

In [107]:
import os

In [108]:
def lee_imagen(path):
  return cv2.imread(path)

In [109]:
def imagen_color_a_grises(im):
  return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

In [110]:
def binariza_imagen(im):
  return cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

In [111]:
def invierte_imagen(im):
  return cv2.bitwise_not(im)

In [112]:
def morf_dilatacion(im, kernel=np.ones((2, 2), np.uint8)):
  return cv2.dilate(im, kernel=kernel)

In [113]:
def guarda_imagen(im, path):
  return Image.fromarray(im, 'L').save(path)

En esta etapa se binariza las imagenes

ruta_lectura: Ruta de la carpeta de las imagenes a binarizar

ruta_escritura: Ruta de la carpeta de las imagenes procesadas



In [114]:
def obtener_ruta(ruta_lectura,archivo):
    return f'{ruta_lectura}{archivo}'

In [115]:
def obtener_archivos(ruta):
   return os.listdir(ruta)

In [116]:
def procesar_archivo(ruta_lectura, ruta_escritura, archivo):
    ruta_lectura_archivo = obtener_ruta(ruta_lectura,archivo)
    captcha = lee_imagen(ruta_image)
    captcha_gris = imagen_color_a_grises(captcha)
    captcha_bn = binariza_imagen(captcha_gris)
    captcha_inv = invierte_imagen(captcha_bn)
    captcha_final = morf_dilatacion(captcha_inv)
    ruta_escritura_archivo = obtener_ruta(ruta_escritura,archivo)
    guarda_imagen(captcha_final, ruta_escritura_archivo)

In [117]:
def main_process_transformacion(ruta_lectura, ruta_escritura):
  archivos = obtener_archivos(ruta_lectura)
  for archivo in archivos:
    procesar_archivo(ruta_lectura, ruta_escritura, archivo)

In [118]:
main_process_transformacion("./data/tagged/", "./data/binary/")

In [119]:
def conv2d(arr, kernel):
    s = kernel.shape + tuple(np.subtract(arr.shape, kernel.shape) + 1)
    strd = np.lib.stride_tricks.as_strided
    subM = strd(arr, shape = s, strides = arr.strides * 2)
    return np.einsum('ij,ijkl->kl', kernel, subM)

In [120]:
MAX_VALUE = 255
def normalizar_img(img):
    return img/MAX_VALUE

In [121]:
def convmap(captcha, template):
    t = normalizar_img(template)
    return conv2d(normalizar_img(template), (t-t.mean()))

In [122]:
def locate_best_match(captcha, template, threshold=.9):
    similarities = convmap(captcha, template)
    _, matches = np.where(similarities > MAX_VALUE*threshold)
    while len(matches) == 0:
        _, matches = np.where(similarities > MAX_VALUE*threshold)
        if len(matches) == 0:
            threshold -= .02
            print(f'No matches found, reducing threshold to {threshold}')
    center = int(np.median(matches))
    _, template_size = template.shape
    return (center, center+template_size)

In [123]:
def top_n_score(captcha, template, n=5):
    similarities = convmap(captcha, template)
    return np.sort(similarities.flatten())[-n:].mean() / MAX_VALUE

In [124]:
def template_match(captcha, templates, n=5):
  scores = {}
  for (letra, kernel) in templates:
    scores[letra] = top_n_score(captcha, kernel, n=n)
  return scores

In [125]:
def lee_imagen_bn(ruta):
  return cv2.imread(ruta, 0)

In [126]:
def obtener_nombre_archivo(archivo):
    return archivo.split('.')[0]

In [127]:
def procesar_templates (ruta_templates):
    templates = obtener_archivos(ruta_templates)
    templates_dict = {}
    for template in templates:
        key = obtener_nombre_archivo(template)
        ruta = obtener_ruta(ruta_templates, template)
        templates_dict[key] = lee_imagen_bn(ruta)
    return  templates_dict

In [128]:
def procesar_captchas(ruta_templates, ruta_captcha, templates_dict):
    archivos = obtener_archivos(ruta_captcha)
    templates = obtener_archivos(ruta_templates)
    captcha_dict = {}
    captcha_scores = {}
    for archivo in archivos:
        key = obtener_nombre_archivo(archivo)
        ruta = obtener_ruta(ruta_captcha, archivo)
        captcha = lee_imagen_bn(ruta)
        captcha_dict[key] = captcha
        captcha_scores[key] = template_match(captcha, templates_dict.items())
    return captcha_dict, captcha_scores

In [129]:
def main_process_consulta(ruta_templates, ruta_captcha):
    templates_dict = procesar_templates(ruta_templates)
    captcha_dict,  captcha_scores = procesar_captchas(
        ruta_templates, ruta_captcha, templates_dict)
    return captcha_dict, templates_dict, captcha_scores


In [130]:
results = main_process_consulta('./data/templates/', './data/to_classify/')

In [131]:
results[2]

{'2bfpx_0001': {'2': 1.413743578543247,
  '3': 1.3277712451453503,
  '4': 1.3938054725969256,
  '5': 1.3867042139485772,
  '6': 1.265008284897964,
  'b': 1.4189436853327075,
  'd': 1.5762765317113159,
  'e': 1.0323330853917012,
  'k': 1.225807495158801,
  'p': 1.4594693975061914,
  'w': 1.5087686416676787},
 '2c5k5_0001': {'2': 1.413743578543247,
  '3': 1.3277712451453503,
  '4': 1.3938054725969256,
  '5': 1.3867042139485772,
  '6': 1.265008284897964,
  'b': 1.4189436853327075,
  'd': 1.5762765317113159,
  'e': 1.0323330853917012,
  'k': 1.225807495158801,
  'p': 1.4594693975061914,
  'w': 1.5087686416676787},
 '2dga8_0001': {'2': 1.413743578543247,
  '3': 1.3277712451453503,
  '4': 1.3938054725969256,
  '5': 1.3867042139485772,
  '6': 1.265008284897964,
  'b': 1.4189436853327075,
  'd': 1.5762765317113159,
  'e': 1.0323330853917012,
  'k': 1.225807495158801,
  'p': 1.4594693975061914,
  'w': 1.5087686416676787},
 '2ek2m_0001': {'2': 1.413743578543247,
  '3': 1.3277712451453503,
  '4':